In [9]:
import os
import tensorflow as tf
from gan.CGAN import CGanNet
import tensorflow_datasets as tfds
import numpy as np

In [10]:
BATCH_SIZE = 32
IMAGE_WIDTH = 112
IMAGE_HEIGHT = 112

In [11]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [12]:
# Load and prepare dataset

(training_set, validation_set), dataset_info = tfds.load(
    'oxford_flowers102',
    split=['test', 'train'],
    with_info=True,
    as_supervised=True,
)

num_training_examples = 0
for example in training_set:
    num_training_examples += 1

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))/255.0
    return image, label

train_batches = training_set.shuffle(num_training_examples//4).map(format_image)

num_classes = dataset_info.features['label'].num_classes
train_batches_images = np.array([_[0] for _ in train_batches])
train_batches_labels = np.array([_[1] for _ in train_batches])

In [13]:
cgan = CGanNet(batch_size=BATCH_SIZE,
          batches_per_epoch=40,
          image_width=IMAGE_WIDTH,
          image_height=IMAGE_HEIGHT,
          number_of_channels=3,
          latent_dimension=100,
          training_data=train_batches_images,
          labels_data=train_batches_labels,
          number_of_classes=num_classes
)

In [14]:
#cgan.clear_files_structure()

In [15]:
cgan.define_discriminator()
cgan.define_generator()
cgan.define_gan()
cgan.create_files_structure()

In [ ]:
#cgan.train(number_of_epochs=1, load_past_model=False)
#cgan.plot_loss()

In [ ]:
cgan.train(number_of_epochs=100, load_past_model=True)
cgan.plot_loss()